# COX REGRESSION (sklearn adapter)

In [2]:
# BEST data (98 patients)
import pandas as pd

radiomics_df = pd.read_excel("survival_radiomics.xlsx", engine='openpyxl')
target_df = pd.read_excel("db_basis_survival.xlsx", engine='openpyxl')

common_indices = radiomics_df['PP']

## EHR

In [11]:
directory = r"L:\basic\divi\jstoker\slicer_pdac\Master Students SS 23\Mattia\survival.xlsx"
ehr = pd.read_excel(directory, usecols='B,C,H,I,J,K,L,D', nrows=137).iloc[1:,:]

#change to int
ehr['Randomisatie nummer'] = ehr['Randomisatie nummer'].astype('int')

#create new_column_id
ehr['PP'] = ehr['Studie'] + '-' + ehr['Randomisatie nummer'].astype(str)

#drop unecessary columnns
ehr = ehr.drop(['Randomisatie nummer', 'Studie'], axis=1)

ehr = ehr.set_index('PP')

ehr = ehr.loc[common_indices]

new_columns = {ehr.columns[0]: 'sex', ehr.columns[1]: 'age', ehr.columns[2]: 'tumor diameter', ehr.columns[3]: 'tumor location', ehr.columns[4]: 'resection margin', ehr.columns[5]: 'nat'}
ehr = ehr.rename(columns = new_columns)

ehr['sex'] = (ehr['sex'] - 1).astype(int)

ehr

,sex,age,tumor diameter,tumor location,resection margin,nat
PP,,,,,,
PP1-125,0,63.468493,3.5,1,none,3
PP1-136,0,68.898630,3.5,1,R0,1
PP1-137,1,59.961644,2.7,1,R1,3
PP1-145,1,68.693151,4.0,4,R0,3
PP1-15,1,73.613699,3.0,1,R0,3
...,...,...,...,...,...,...
PP2-77,1,63.106849,3.5,4,R1,2
PP2-8,1,66.153425,NaN,1,R0,3
PP2-87,0,54.260274,4.7,2,R0,3


#### tumor location

In [12]:
#change 2.6 with mode
mode = int(ehr['tumor location'].mode().values)
ehr.loc['PP2-321', 'tumor location'] = mode
ehr.loc['PP2-321']

sex                         1
age                 60.380822
tumor diameter            1.0
tumor location              1
resection margin           R0
nat                         2
Name: PP2-321, dtype: object

In [13]:
ehr['tumor location'].unique()

array([1, 4, '1 *Groove', 1.3, 2, 3, 1.2], dtype=object)

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Replacing '1 *Groove' with 1
ehr['tumor location'] = ehr['tumor location'].replace('1 *Groove', 1)

# Creating two new columns '2' and '3' for categories 2 and 3
ehr['2'] = 0
ehr['3'] = 0

# Update these columns whenever we encounter 1.2 or 1.3
ehr.loc[ehr['tumor location'] == 1.2, '2'] = 1
ehr.loc[ehr['tumor location'] == 1.3, '3'] = 1

# Also replacing 1.2 and 1.3 with 1
ehr['tumor location'] = ehr['tumor location'].replace([1.2, 1.3], 1)

# One-hot-encode the 'tumor location'
onehotencoder = OneHotEncoder()
ehr_encoded = onehotencoder.fit_transform(ehr['tumor location'].values.reshape(-1,1)).toarray()

# Creating a DataFrame from our encoded array
ehr_encoded = pd.DataFrame(ehr_encoded, columns = ["tumor_location_"+str(int(i)) for i in range(ehr_encoded.shape[1])])

# set same index also for ehr_encoded
ehr_encoded = ehr_encoded.set_index(ehr.index)

# Concatenating the original DataFrame and the one-hot-encoded DataFrame
ehr = pd.concat([ehr, ehr_encoded], axis=1)

#adding values from column 2 and 3
ehr['tumor_location_2'] = ehr['tumor_location_2'] + ehr['2']
ehr['tumor_location_3'] = ehr['tumor_location_3'] + ehr['3']

# Dropping the '2' and '3' columns
ehr = ehr.drop(columns=['2', '3', 'tumor location'])

#### resection margin

In [15]:
ehr['resection margin'].unique()

array(['none', 'R0', 'R1'], dtype=object)

In [16]:
onehotencoder = OneHotEncoder()
ehr_encoded = onehotencoder.fit_transform(ehr['resection margin'].values.reshape(-1,1)).toarray()

# Creating a DataFrame from our encoded array
ehr_encoded = pd.DataFrame(ehr_encoded, columns = ["resection_margin_"+str(int(i)) for i in range(ehr_encoded.shape[1])])

# set same index also for ehr_encoded
ehr_encoded = ehr_encoded.set_index(ehr.index)

# Concatenating the original DataFrame and the one-hot-encoded DataFrame
ehr = pd.concat([ehr, ehr_encoded], axis=1)

# drop
ehr = ehr.drop(columns=['resection margin'])

#### nat

In [17]:
ehr['nat'].unique()

array([3, 1, 2, nan], dtype=object)

In [18]:
# Calculate the mode of the 'nat' column
mode_value = ehr['nat'].mode()[0]

# Fill NaN values with the mode value
ehr['nat'] = ehr['nat'].fillna(mode_value)

onehotencoder = OneHotEncoder()
ehr_encoded = onehotencoder.fit_transform(ehr['nat'].values.reshape(-1,1)).toarray()

# Creating a DataFrame from our encoded array
ehr_encoded = pd.DataFrame(ehr_encoded, columns = ["nat_"+str(int(i)) for i in range(ehr_encoded.shape[1])])

# set same index also for ehr_encoded
ehr_encoded = ehr_encoded.set_index(ehr.index)

# Concatenating the original DataFrame and the one-hot-encoded DataFrame
ehr = pd.concat([ehr, ehr_encoded], axis=1)

# drop
ehr = ehr.drop(columns=['nat'])

In [19]:
ehr.head()

,sex,age,tumor diameter,tumor_location_0,tumor_location_1,tumor_location_2,tumor_location_3,resection_margin_0,resection_margin_1,resection_margin_2,nat_0,nat_1,nat_2
PP,,,,,,,,,,,,,
PP1-125,0,63.468493,3.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
PP1-136,0,68.898630,3.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
PP1-137,1,59.961644,2.7,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
PP1-145,1,68.693151,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
PP1-15,1,73.613699,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [20]:
#WEIIIIIRDDDDDD
ehr[ehr['nat_0']==1]

,sex,age,tumor diameter,tumor_location_0,tumor_location_1,tumor_location_2,tumor_location_3,resection_margin_0,resection_margin_1,resection_margin_2,nat_0,nat_1,nat_2
PP,,,,,,,,,,,,,
PP1-136,0,68.898630,3.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
PP1-150,0,55.838356,2.8,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
PP1-16,0,67.109589,3.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
PP1-74,0,64.487671,3.7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
PP1-96,1,56.224658,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
PP2-164,0,76.076712,4.2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
PP2-21,0,73.887671,2.9,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
PP2-25,1,63.178082,3.4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
PP2-28,1,63.106849,4.3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [21]:
ehr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, PP1-125 to PP2-98
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sex                 98 non-null     int32  
 1   age                 98 non-null     float64
 2   tumor diameter      97 non-null     float64
 3   tumor_location_0    98 non-null     float64
 4   tumor_location_1    98 non-null     float64
 5   tumor_location_2    98 non-null     float64
 6   tumor_location_3    98 non-null     float64
 7   resection_margin_0  98 non-null     float64
 8   resection_margin_1  98 non-null     float64
 9   resection_margin_2  98 non-null     float64
 10  nat_0               98 non-null     float64
 11  nat_1               98 non-null     float64
 12  nat_2               98 non-null     float64
dtypes: float64(12), int32(1)
memory usage: 12.4+ KB


#### tumor diamater

In [22]:
# Calculate the mean of the 'nat' column
mean_value = ehr['tumor diameter'].mean()

# Fill NaN values with the mode value
ehr['tumor diameter'] = ehr['tumor diameter'].fillna(mean_value)

### Concatenate with dataset

In [23]:
merged_df = pd.concat([ehr, target_df.set_index('PP')], axis=1)

In [24]:
merged_df.head()

,sex,age,tumor diameter,tumor_location_0,tumor_location_1,tumor_location_2,tumor_location_3,resection_margin_0,resection_margin_1,resection_margin_2,nat_0,nat_1,nat_2,Event,Duration
PP,,,,,,,,,,,,,,,
PP1-125,0,63.468493,3.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1,91
PP1-136,0,68.898630,3.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,58
PP1-137,1,59.961644,2.7,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,1479
PP1-145,1,68.693151,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1,1627
PP1-15,1,73.613699,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0,3489


In [25]:
merged_df.shape

(98, 15)

## Modeling with Train and Test splits

### variance threshold, scaling, pca

#### only on one test split

#### repeated cross validation

In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from lifelines.utils.sklearn_adapter import sklearn_adapter
from lifelines import CoxPHFitter
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

X = merged_df.drop('Duration', axis=1)  # keep as a dataframe
y = merged_df['Duration']

# Define the number of folds and repeats, and an empty list to store the scores
num_folds = 5
num_repeats = 10
scores = []

for r in range(num_repeats):
    print(f"--- Repeat: {r + 1} ---")
    
    # Generate indices for splits
    np.random.seed(r+420)  # change the seed for each repeat
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    fold_sizes = (len(X) // num_folds) * np.ones(num_folds, dtype=int)  # equally divide indices
    fold_sizes[:len(X) % num_folds] += 1  # if len(X) is not exactly divisible by num_folds, assign remainder to first few

    current = 0
    splits = []
    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        splits.append((indices[start:stop], np.concatenate((indices[:start], indices[stop:]))))  # (test, train)
        current = stop

    # Perform cross-validation
    for i, (test_idx, train_idx) in enumerate(splits):
        print(f"Fold: {i + 1}")
        
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Drop 'Event' column
        X_train_new = X_train.drop('Event', axis=1)
        X_test_new = X_test.drop('Event', axis=1)
        
        #change to array
        X_train_new = X_train_new.values
        X_test_new = X_test_new.values
        
        # Scale the data
        scaler = StandardScaler()
        scaler = MinMaxScaler()
        X_train_new[:, :2] = scaler.fit_transform(X_train_new[:, :2])
        X_test_new[:, :2] = scaler.transform(X_test_new[:, :2])
        
        # Convert back to DataFrame
        X_train_new = pd.DataFrame(X_train_new)
        X_test_new = pd.DataFrame(X_test_new)
        
        # Add back indices
        X_train_new.index = X_train.index
        X_test_new.index = X_test.index
        
        # Add 'Event' column back
        X_train_new['Event'] = X_train['Event']
        X_test_new['Event'] = X_test['Event']
        
        # Instantiate CoxRegression object
        CoxRegression = sklearn_adapter(CoxPHFitter, event_col='Event')
        sk_cph = CoxRegression(penalizer=1e-5)
        sk_cph.fit(X_train_new, y_train)
        
        # Calculate and store the score
        score = sk_cph.score(X_test_new, y_test)
        scores.append(score)

        print(f"Score: {score}")
        print("-----------------------------")

# Calculate mean
mean_score = np.mean(scores)

print(f"Mean Score: {mean_score}")

--- Repeat: 1 ---
Fold: 1
Score: 0.6046511627906976
-----------------------------
Fold: 2
Score: 0.60625
-----------------------------
Fold: 3
Score: 0.6804733727810651
-----------------------------
Fold: 4
Score: 0.7769784172661871
-----------------------------
Fold: 5
Score: 0.7707006369426752
-----------------------------
--- Repeat: 2 ---
Fold: 1
Score: 0.7375
-----------------------------
Fold: 2
Score: 0.5842696629213483
-----------------------------
Fold: 3
Score: 0.6196319018404908
-----------------------------
Fold: 4
Score: 0.7019867549668874
-----------------------------
Fold: 5
Score: 0.7142857142857143
-----------------------------
--- Repeat: 3 ---
Fold: 1
Score: 0.6402439024390244
-----------------------------
Fold: 2
Score: 0.7467532467532467
-----------------------------
Fold: 3
Score: 0.7906976744186046
-----------------------------
Fold: 4
Score: 0.6602564102564102
-----------------------------
Fold: 5
Score: 0.6118421052631579
-----------------------------
--- Repea

# Random Forest Survival

#### 10 different cv with 5 folds

In [27]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sksurv.ensemble import RandomSurvivalForest
from sklearn.decomposition import PCA

X = merged_df.drop(['Duration', 'Event'], axis=1)  # keep as a dataframe
y = merged_df[['Event','Duration']]

#change y to array of tuples (Event, Duration)
y = np.array([(bool(arr[0]), arr[1]) for arr in y.values], dtype=[('boolean', bool), ('integer', int)])

# Define the number of folds, repeats and an empty list to store the scores
num_folds = 5
num_repeats = 10
scores = []

# Create the RepeatedKFold object
rkf = RepeatedKFold(n_splits=num_folds, n_repeats=num_repeats, random_state=1)

# Perform Repeated K-Fold cross-validation
for i, (train_index, test_index) in enumerate(rkf.split(X)):
    print(f"Fold: {(i % num_folds) + 1}, Repeat: {i // num_folds + 1}")
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
     #change to array
    X_train_new = X_train.values
    X_test_new = X_test.values
    
    # Scale the data
    scaler = StandardScaler()
    scaler = MinMaxScaler()
    X_train_new[:, :2] = scaler.fit_transform(X_train_new[:, :2])
    X_test_new[:, :2] = scaler.transform(X_test_new[:, :2])
    
    # Convert back to DataFrame
    X_train_new = pd.DataFrame(X_train_new)
    X_test_new = pd.DataFrame(X_test_new)
    
    # Add back indices
    X_train_new.index = X_train.index
    X_test_new.index = X_test.index
    
    # Instantiate RandomSurvivalForest object
    rsf = RandomSurvivalForest()
    rsf.fit(X_train_new, y_train)
    
    # Calculate and store the score
    score = rsf.score(X_test_new, y_test)
    scores.append(score)
    
    print(f"Score: {score}")
    print("-----------------------------")

# Calculate mean and 95% confidence interval of the scores
mean_score = np.mean(scores)

print(f"Mean Score: {mean_score}")

Fold: 1, Repeat: 1
Score: 0.696551724137931
-----------------------------
Fold: 2, Repeat: 1
Score: 0.54
-----------------------------
Fold: 3, Repeat: 1
Score: 0.7150537634408602
-----------------------------
Fold: 4, Repeat: 1
Score: 0.5506329113924051
-----------------------------
Fold: 5, Repeat: 1
Score: 0.7612903225806451
-----------------------------
Fold: 1, Repeat: 2
Score: 0.783625730994152
-----------------------------
Fold: 2, Repeat: 2
Score: 0.7595628415300546
-----------------------------
Fold: 3, Repeat: 2
Score: 0.5698324022346368
-----------------------------
Fold: 4, Repeat: 2
Score: 0.527027027027027
-----------------------------
Fold: 5, Repeat: 2
Score: 0.8620689655172413
-----------------------------
Fold: 1, Repeat: 3
Score: 0.7714285714285715
-----------------------------
Fold: 2, Repeat: 3
Score: 0.6777777777777778
-----------------------------
Fold: 3, Repeat: 3
Score: 0.6962025316455697
-----------------------------
Fold: 4, Repeat: 3
Score: 0.64705882352941

# GDB survival (ComponentwiseGradientBoostingSurvivalAnalysis)

#### 10 different cv with 5 folds

In [28]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sklearn.decomposition import PCA

X = merged_df.drop(['Duration', 'Event'], axis=1)  # keep as a dataframe
y = merged_df[['Event','Duration']]

#change y to array of tuples (Event, Duration)
y = np.array([(bool(arr[0]), arr[1]) for arr in y.values], dtype=[('boolean', bool), ('integer', int)])

# Define the number of folds, repeats and an empty list to store the scores
num_folds = 5
num_repeats = 10
scores = []

# Create the RepeatedKFold object
rkf = RepeatedKFold(n_splits=num_folds, n_repeats=num_repeats, random_state=1)

# Perform Repeated K-Fold cross-validation
for i, (train_index, test_index) in enumerate(rkf.split(X)):
    print(f"Fold: {(i % num_folds) + 1}, Repeat: {i // num_folds + 1}")
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #change to array
    X_train_new = X_train.values
    X_test_new = X_test.values
    
    # Scale the data
    scaler = StandardScaler()
    scaler = MinMaxScaler()
    X_train_new[:, :2] = scaler.fit_transform(X_train_new[:, :2])
    X_test_new[:, :2] = scaler.transform(X_test_new[:, :2])
    
    # Convert back to DataFrame
    X_train_new = pd.DataFrame(X_train_new)
    X_test_new = pd.DataFrame(X_test_new)
    
    # Add back indices
    X_train_new.index = X_train.index
    X_test_new.index = X_test.index
    
    # Instantiate GB object
    gdb = ComponentwiseGradientBoostingSurvivalAnalysis(loss="coxph")
    gdb.fit(X_train_new, y_train)
    
    # Calculate and store the score
    score = gdb.score(X_test_new, y_test)
    scores.append(score)
    
    print(f"Score: {score}")
    print("-----------------------------")

# Calculate mean and 95% confidence interval of the scores
mean_score = np.mean(scores)

print(f"Mean Score: {mean_score}")

Fold: 1, Repeat: 1
Score: 0.6482758620689655
-----------------------------
Fold: 2, Repeat: 1
Score: 0.6166666666666667
-----------------------------
Fold: 3, Repeat: 1
Score: 0.6397849462365591
-----------------------------
Fold: 4, Repeat: 1
Score: 0.5443037974683544
-----------------------------
Fold: 5, Repeat: 1
Score: 0.7064516129032258
-----------------------------
Fold: 1, Repeat: 2
Score: 0.6842105263157895
-----------------------------
Fold: 2, Repeat: 2
Score: 0.7213114754098361
-----------------------------
Fold: 3, Repeat: 2
Score: 0.5865921787709497
-----------------------------
Fold: 4, Repeat: 2
Score: 0.5337837837837838
-----------------------------
Fold: 5, Repeat: 2
Score: 0.7672413793103449
-----------------------------
Fold: 1, Repeat: 3
Score: 0.7257142857142858
-----------------------------
Fold: 2, Repeat: 3
Score: 0.6166666666666667
-----------------------------
Fold: 3, Repeat: 3
Score: 0.6740506329113924
-----------------------------
Fold: 4, Repeat: 3
Score: